### Import modules used for this project

In [55]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
import requests # library to handle requests

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library
from folium import plugins
print('Libraries imported.')

Libraries imported.


### Retrieve geographical coordinates for the city of Portland

In [56]:
address = 'Portland'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Portland are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Portland are 45.5202471, -122.6741949.


### Create dataframe based on crime data. Drop unnecessary rows. Ensure columns are type 'string'

In [57]:
df = pd.read_csv('crimedata018.csv')
df = df.dropna()
df = df.dropna(axis=0, how='any')
df.columns=list(map(str, df.columns))

### Use value_counts to get the number of crimes for each neighborhood

In [58]:
nList = df['Neighborhood'].value_counts().index
cList = df['Neighborhood'].value_counts().values
data = {'Neighborhood': nList, 'Count': cList}
zf = pd.DataFrame(data)
zf = zf.dropna()
zf

,Count,Neighborhood
0,3127,Hazelwood
1,2751,Downtown
2,1873,Lents
3,1786,Powellhurst-Gilbert
4,1602,Centennial
5,1225,Montavilla
6,1208,Northwest
7,1179,Lloyd
8,1056,Pearl
9,923,St Johns


### Merge original dataframe with new counts for each neighborhood

In [59]:
mergedDF = pd.merge(zf, df, how ='outer', on ='Neighborhood') 
mergedDF.head()

,Count,Neighborhood,Address,CaseNumber,CrimeAgainst,OccurDate,OccurTime,OffenseCategory,OffenseType,OpenDataLat,OpenDataLon,OpenDataX,OpenDataY,ReportDate,OffenseCount
0,3127,Hazelwood,100 BLOCK OF NE 97TH AVE,18-223178,Property,7/2/2018,2030,Larceny Offenses,Theft From Motor Vehicle,45.523703,-122.563671,7673250.0,683784.0,7/3/2018,1
1,3127,Hazelwood,100 BLOCK OF NE 99TH AVE,18-76681,Property,3/5/2018,1100,Fraud Offenses,Identity Theft,45.523761,-122.562311,7673599.0,683796.0,3/7/2018,1
2,3127,Hazelwood,100 BLOCK OF NE 102ND AVE,18-145489,Property,5/3/2018,30,Vandalism,Vandalism,45.523437,-122.558318,7674619.0,683652.0,5/3/2018,1
3,3127,Hazelwood,100 BLOCK OF NE 102ND AVE,18-171942,Person,5/23/2018,2022,Assault Offenses,Simple Assault,45.523437,-122.558318,7674619.0,683652.0,5/23/2018,1
4,3127,Hazelwood,100 BLOCK OF NE 102ND AVE,18-178840,Society,5/29/2018,1201,Animal Cruelty Offenses,Animal Cruelty,45.523437,-122.558318,7674619.0,683652.0,5/29/2018,1


In [60]:
mergedDF = mergedDF.dropna()
mergedDF = mergedDF.dropna(axis=0, how='any')

### Change name of Buckman neighborhoods to be more data consistent

In [61]:
for i in range(0, len(mergedDF['Neighborhood'])):
    if mergedDF['Neighborhood'][i] == 'Buckman West' or mergedDF['Neighborhood'][i] == 'Buckman East':
        mergedDF['Neighborhood'][i] = 'Buckman'

C:\Users\Dustin\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


### Create a choropleth map based on the neighborhood GeoJSON file combined with the crime data.

In [62]:
world_geo = r'Neighborhood_Boundaries.geojson' # geojson file

threshold_scale = np.linspace(0,
                              3999,
                              10, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 

world_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# generate choropleth map using the total immigration of each country to Canada from 1980 to 2013
folium.Choropleth(
    geo_data=world_geo,
    name = 'choropleth',
    data=mergedDF,
    columns=['Neighborhood', 'Count'],
    key_on='feature.properties.MAPLABEL',
    threshold_scale = threshold_scale,
    fill_color='YlOrRd',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Crime in Portland'
).add_to(world_map)


### This map reveals that downtown Portland as well as parts of east Portland are crime heavy. 
### West Portland has less crime. Parts of North/Northwest Portland have fewer crimes, but St Johns is an exception

In [63]:
world_map

### Acquire bike shop data for the city

In [64]:
bicycleData = pd.read_csv(r'Recommended_Bicycle_Route_Points.csv')

bikeDF = bicycleData.loc[bicycleData['Type']=='BIKESHOP']

bikeDF

,X,Y,OBJECTID,TranPlanID,Type,Description,Rotation,LinkPath
147,-122.666459,45.548597,148,TP08-0000626,BIKESHOP,Abraham Fixes Bikes,90.0,http://abrahamfixesbikes.com/
148,-122.589576,45.482851,149,TP08-0000628,BIKESHOP,BackPedal Cycle Works,90.0,http://www.bpcycleworks.com/
149,-122.855378,45.636380,150,TP08-0000629,BIKESHOP,Bike Central,90.0,http://www.bike-central.com
150,-122.684001,45.517997,151,TP08-0000631,BIKESHOP,The Bike Gallery - Downtown,90.0,http://www.bikegallery.com/about/downtown-pg62...
151,-122.608537,45.540900,152,TP08-0000632,BIKESHOP,The Bike Gallery - Hollywood,90.0,http://www.bikegallery.com/about/hollywood-pg6...
152,-122.648927,45.473944,153,TP08-0000633,BIKESHOP,The Bike Gallery - Westmoreland,90.0,http://www.bikegallery.com/about/woodstock-pg6...
153,-122.573519,45.564961,154,TP08-0000634,BIKESHOP,Bike Tires Direct,90.0,http://www.biketiresdirect.com/
154,-122.630147,45.496386,155,TP08-0000635,BIKESHOP,Bikes For Humanity PDX,90.0,http://b4hpdx.org/
155,-122.755002,45.589317,156,TP08-0000637,BIKESHOP,Block Bikes,90.0,http://blockbikespdx.com/
156,-122.684853,45.463662,157,TP08-0000638,BIKESHOP,Burlingame Bikes,90.0,http://www.burlingamebikes.com/


In [65]:
bp = folium.map.FeatureGroup()

### Create visible markers to give an idea where in Portland bike shops are located

In [66]:
for lat, lng, label in zip(bikeDF.Y, bikeDF.X, bikeDF.Description):
    bp.add_child(
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='white',
        fill_opacity=0.6
    )
)

In [67]:
world_map.add_child(bp)

### After the client has narrowed down the neighborhoods, Foursquare API is used to learn more about the surrounding neighborhood venues

In [68]:
CLIENT_ID = 'RYRDZFDO1ODCLXGJDLJ5LIJQMTUDPSQX2WVOK5IIUXF1DMDZ' # your Foursquare ID
CLIENT_SECRET = 'K3QEVOZHFDLRY3VVBESZA5FQIWSAQZOYD03L03NTPCZZD5PK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

### Function to loop GET requests

In [69]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Dataframe created linking points of interest to neighborhoods which the client wishes to pursue.

In [70]:
bf = {
    'Neighborhood':['Multnomah','Burlingame', 'Woodstock', 'Brooklyn', 'Mt Tabor', 'Buckman', 'Buckman', 'Buckman','Buckman','Buckman','Buckman'],
    'Description':['Southwest Bicycle','Burlingame Bikes', 'The Missing Link', 'TomCat Bikes', 'Mount Tabor Cyclery', 'Recumbent PDX', 'River City Bicyles','evo','Citybikes Workers Cooperative','Universle Cycles','Crank'],
    'Latitude':['45.467571', '45.463662', '45.479425', '45.500505', '45.522232', '45.512415', '45.516298','45.521226','45.521983','45.522465','45.522404'],
    'Longitude':['-122.714317', '-122.684853', '-122.614736', '-122.654548', '-122.606919', '-122.645229', '-122.659965', '-122.661435','-122.646118','-122.642937','-122.638097']
}

bf = pd.DataFrame(bf)
bf

,Description,Latitude,Longitude,Neighborhood
0,Southwest Bicycle,45.467571,-122.714317,Multnomah
1,Burlingame Bikes,45.463662,-122.684853,Burlingame
2,The Missing Link,45.479425,-122.614736,Woodstock
3,TomCat Bikes,45.500505,-122.654548,Brooklyn
4,Mount Tabor Cyclery,45.522232,-122.606919,Mt Tabor
5,Recumbent PDX,45.512415,-122.645229,Buckman
6,River City Bicyles,45.516298,-122.659965,Buckman
7,evo,45.521226,-122.661435,Buckman
8,Citybikes Workers Cooperative,45.521983,-122.646118,Buckman
9,Universle Cycles,45.522465,-122.642937,Buckman


### Foursquare returns venue information for the selected neighborhoods. 

In [71]:
bike_venues = getNearbyVenues(names=bf['Neighborhood'],
                                   latitudes=bf['Latitude'],
                                   longitudes=bf['Longitude']
                                  )
bike_venues

Multnomah
Burlingame
Woodstock
Brooklyn
Mt Tabor
Buckman
Buckman
Buckman
Buckman
Buckman
Buckman


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Multnomah,45.467571,-122.714317,Annie Bloom's Books,45.467684,-122.713569,Bookstore
1,Multnomah,45.467571,-122.714317,Nectar Frozen Yogurt,45.467830,-122.713895,Frozen Yogurt Shop
2,Multnomah,45.467571,-122.714317,John's Marketplace,45.467385,-122.713203,Food & Drink Shop
3,Multnomah,45.467571,-122.714317,Marco's Cafe & Espresso Bar,45.467377,-122.712827,Café
4,Multnomah,45.467571,-122.714317,Grand Central Baking Company,45.467219,-122.711979,Bakery
5,Multnomah,45.467571,-122.714317,Thinker Toys,45.467912,-122.712727,Toy / Game Store
6,Multnomah,45.467571,-122.714317,Fat City Cafe,45.467906,-122.713284,Breakfast Spot
7,Multnomah,45.467571,-122.714317,Journeys,45.468370,-122.712240,Bar
8,Multnomah,45.467571,-122.714317,The Ship Tavern,45.467686,-122.712960,Dive Bar
9,Multnomah,45.467571,-122.714317,Annastasia Salon,45.468530,-122.711457,Cosmetics Shop


### Isolate competitors based on venue type

In [72]:
competitors = [
    
    'Food & Drink Shop',
    'Café',
    'Bakery',
    'Breakfast Spot',
    'Coffee Shop',
    'Donut Shop',
    'Dessert Shop',
    'Chocolate Shop'
]

In [73]:
competitorDF = pd.DataFrame(columns = bike_venues.columns)
for comp in competitors:
    x = bike_venues.loc[bike_venues['Venue Category']==comp]
    competitorDF = pd.concat([competitorDF, x])

### Competitor dataframe shows frequency and brand of competition by each neighborhood, close to original bicycle shop data point

In [74]:
competitorDF.sort_values(['Neighborhood'])

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
90,Brooklyn,45.500505,-122.654548,Pine State Biscuits,45.504686,-122.654599,Breakfast Spot
87,Brooklyn,45.500505,-122.654548,Sanborn's,45.500255,-122.654037,Breakfast Spot
84,Brooklyn,45.500505,-122.654548,Southeast Grind,45.500973,-122.652556,Café
88,Brooklyn,45.500505,-122.654548,Warehouse Cafe,45.497781,-122.653855,Café
102,Brooklyn,45.500505,-122.654548,Ford Food and Drink,45.504763,-122.654911,Coffee Shop
96,Brooklyn,45.500505,-122.654548,Genies Cafe,45.504973,-122.654345,Breakfast Spot
415,Buckman,45.522404,-122.638097,Alma Chocolate,45.524200,-122.637197,Chocolate Shop
411,Buckman,45.522404,-122.638097,Crema Bakery and Cafe,45.522242,-122.637039,Coffee Shop
379,Buckman,45.522465,-122.642937,Crema Bakery and Cafe,45.522242,-122.637039,Coffee Shop
386,Buckman,45.522465,-122.642937,Alma Chocolate,45.524200,-122.637197,Chocolate Shop


### Based on this information, our client decides to build her business in the Mt Tabor neighborhood.